In [1]:
import pandas as pd
import re
from pathlib import Path

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
data_folder = Path("/content/drive/MyDrive/Airline")

In [4]:
aa_otp = pd.read_csv(data_folder/"AA_OTP.csv")
alaska_otp = pd.read_csv(data_folder/"Alaska_OTP.csv")
delta_otp = pd.read_csv(data_folder/"Delta_OTP.csv")
sw_otp = pd.read_csv(data_folder/"SouthWest_OTP.csv")
united_otp = pd.read_csv(data_folder/"United_OTP.csv")
aa = pd.read_excel(data_folder/"AA.xlsx").transpose()
alaska = pd.read_excel(data_folder/"Alaska.xlsx").transpose()
delta = pd.read_excel(data_folder/"Delta.xlsx").transpose()
sw = pd.read_excel(data_folder/"SouthWest.xlsx").transpose()
united = pd.read_excel(data_folder/"United.xlsx").transpose()

In [5]:
def rmvSpaceFromCol(df):
    newCol = []
    cols = df.columns
    for col in cols:
        newCol.append(re.sub(r'\s+','',col))
    df.columns = newCol

In [6]:
rmvSpaceFromCol(aa_otp)
rmvSpaceFromCol(alaska_otp)
rmvSpaceFromCol(delta_otp)
rmvSpaceFromCol(sw_otp)
rmvSpaceFromCol(united_otp)

In [7]:
aa_otp.columns

Index(['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [8]:
aa_otp = aa_otp.iloc[:,[0,6,7]]
alaska_otp = alaska_otp.iloc[:,[0,6,7]]
delta_otp = delta_otp.iloc[:,[0,6,7]]
sw_otp = sw_otp.iloc[:,[0,6,7]]
united_otp = united_otp.iloc[:,[0,6,7]]

In [9]:
print(aa_otp.isna().sum())
print(alaska_otp.isna().sum())
print(delta_otp.isna().sum())
print(sw_otp.isna().sum())
print(united_otp.isna().sum())

year            0
arr_flights     4
arr_del15      11
dtype: int64
year           0
arr_flights    1
arr_del15      2
dtype: int64
year            0
arr_flights    53
arr_del15      63
dtype: int64
year           0
arr_flights    0
arr_del15      0
dtype: int64
year            0
arr_flights    22
arr_del15      35
dtype: int64


In [10]:
%%capture
aa_otp.fillna(aa_otp.median(), inplace=True)
alaska_otp.fillna(alaska_otp.median(), inplace=True)
delta_otp.fillna(delta_otp.median(), inplace=True)
sw_otp.fillna(sw_otp.median(), inplace=True)
united_otp.fillna(united_otp.median(), inplace=True)

In [11]:
aa_otp = aa_otp.groupby('year').sum()
alaska_otp = alaska_otp.groupby('year').sum()
delta_otp = delta_otp.groupby('year').sum()
sw_otp = sw_otp.groupby('year').sum()
united_otp = united_otp.groupby('year').sum()

In [12]:
aa_otp['airline'] = "aa"
alaska_otp['airline'] = "alaska"
delta_otp['airline'] = "delta"
sw_otp['airline'] = "sw"
united_otp['airline'] = "united"

In [13]:
otp = pd.concat([aa_otp, alaska_otp, delta_otp, sw_otp, united_otp])
otp['pc_delay'] = otp['arr_del15'] / otp['arr_flights'] * 100
pc_delay = otp['pc_delay']
otp = otp.drop(columns=['pc_delay'])
otp.insert(loc=2, column='pc_delay', value = pc_delay)
otp.reset_index(inplace=True)

In [14]:
otp = otp[(otp['year'] >= 2006) & (otp['year'] <= 2020)]
otp.columns = ['year','number_of_flights','number_of_delays','percent_delays','airline']

In [15]:
def clean_xlsx(df):
    df = df.reset_index(drop=True)
    df = df.iloc[13:,[1,146,148,159,189]]
    df.columns = ['year','total_fleet_cost','pilot_cost','maintenance_cost','ASM']
    df['total_fleet_cost'] = df['total_fleet_cost'] * 1000
    df['pilot_cost'] = df['pilot_cost'] * 1000
    df['maintenance_cost'] = df['maintenance_cost'] * 1000
    return df

In [16]:
aa = clean_xlsx(aa)
alaska = clean_xlsx(alaska)
delta = clean_xlsx(delta)
sw = clean_xlsx(sw)
united = clean_xlsx(united)

In [17]:
aa['airline'] = "aa"
alaska['airline'] = "alaska"
delta['airline'] = "delta"
sw['airline'] = "sw"
united['airline'] = "united"

In [18]:
cost = pd.concat([aa, alaska, delta, sw, united])

In [19]:
otp = otp.reset_index(drop=True)
cost = cost.reset_index(drop=True)

In [20]:
df = pd.merge(left=cost, right=otp, how="inner", on=['year','airline'])

In [21]:
df['CASM_Total'] = df['total_fleet_cost'] / df['ASM']
df['CASM_Pilot'] = df['pilot_cost'] / df['ASM']
df['CASM_Maintenance'] = df['maintenance_cost'] / df['ASM']

In [22]:
df

,year,total_fleet_cost,pilot_cost,maintenance_cost,ASM,airline,number_of_flights,number_of_delays,percent_delays,CASM_Total,CASM_Pilot,CASM_Maintenance
0,2006,10986061000,1908144000,1976929000,174026191439,aa,643597.0,145563.0,22.617104,0.063129,0.010965,0.01136
1,2007,11210772000,1881909000,1961909000,169910784220,aa,633857.0,178129.0,28.102395,0.06598,0.011076,0.011547
2,2008,13441293000,1824261000,2186197000,163537180890,aa,604885.0,163112.0,26.965787,0.082191,0.011155,0.013368
3,2009,10247163000,1760092000,2226856000,151779066994,aa,551597.0,114591.0,20.774406,0.067514,0.011596,0.014672
4,2010,11146543000,1773128000,2324637000,153245600723,aa,540963.0,99108.0,18.320661,0.072736,0.01157,0.015169
...,...,...,...,...,...,...,...,...,...,...,...,...
70,2016,12934689000,3400886000,2724747000,224652629285,united,545190.0,92865.0,17.033511,0.057576,0.015138,0.012129
71,2017,14380126000,3621185000,2988232000,234546881195,united,584481.0,97603.0,16.699089,0.06131,0.015439,0.01274
72,2018,16450649000,3967806000,2929919000,244770592714,united,621565.0,119668.0,19.252693,0.067208,0.01621,0.01197
73,2019,16326691000,4193601000,2913522000,253254572206,united,626033.0,132160.0,21.110708,0.064468,0.016559,0.011504
